In [1]:
%pip install -q --upgrade keras-nlp
%pip install -q --upgrade keras

%pip install tensorflow
%pip install tensorflow_hub

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached ml_dtypes-0.2.0-cp311-cp311-win_amd64.whl.metadata (20 kB)
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
Using cached ml_dtypes-0.2.0-cp311-cp311-win_amd64.whl (938 kB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
  Attempting uninstall: keras
    Found existing installation: keras 3.0.5
    Uninstalling keras-3.0.5:
      Successfully uninstalled keras-3.0.5
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-nightly-intel 2.17.0.dev20240305 requires ml-dtypes~=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.


In [5]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the dataset with only the needed columns
df = pd.read_csv('twitter.csv', usecols=[2, 3], header=None)
df.columns = ['label', 'text']

# Drop rows with missing values
df.dropna(subset=['label', 'text'], inplace=True)

# Ensure that all text data are strings
df = df[df['text'].apply(lambda x: isinstance(x, str))]

# Filter out rows with invalid labels
valid_labels = ["Positive", "Negative", "Neutral", "Irrelevant"]
df = df[df['label'].isin(valid_labels)]

# Preprocess the text
tweets = df['text']
labels = df['label']

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tweets)
sequences = tokenizer.texts_to_sequences(tweets)

# Pad sequences for uniform length
max_sequence_length = max([len(x) for x in sequences])
X = pad_sequences(sequences, maxlen=max_sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, encoded_labels, test_size=0.2, random_state=42)

Inference with a pretrained classifier

In [8]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics import accuracy_score

# Load a pretrained text classification model from TensorFlow Hub
model_url = 'https://tfhub.dev/google/nnlm-en-dim50/2'  # Example model
hub_layer = hub.KerasLayer(model_url, input_shape=[], dtype=tf.string, trainable=True)

# Build the model
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='softmax'))  # 4 output classes

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Convert integer sequences back to text for the hub model
reverse_word_index = dict([(value, key) for (key, value) in tokenizer.word_index.items()])
def sequence_to_text(sequence):
    return ' '.join([reverse_word_index.get(i, '?') for i in sequence])

X_test_texts = np.array([sequence_to_text(seq) for seq in X_test])

# Make predictions
predictions = model.predict(X_test_texts)

# Process predictions (e.g., taking the argmax for classification)
predicted_classes = np.argmax(predictions, axis=1)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, predicted_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")

463/463 [==============================] - 1s 1ms/step
Accuracy: 29.59%


Fine tuning a pretrained backbone

In [9]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics import accuracy_score

# Load a pretrained text embedding layer from TensorFlow Hub
embedding_url = 'https://tfhub.dev/google/nnlm-en-dim50/2'  # Example embedding layer
hub_layer = hub.KerasLayer(embedding_url, input_shape=[], dtype=tf.string, trainable=True)

# Build the model
model = tf.keras.Sequential([
    hub_layer,
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')  # Assuming 4 output classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Convert sequences to text for training
X_train_texts = np.array([sequence_to_text(seq) for seq in X_train])
X_test_texts = np.array([sequence_to_text(seq) for seq in X_test])

# Train the model with fewer epochs for faster runtime
num_epochs = 2  # Reduced number of epochs
model.fit(X_train_texts, y_train, epochs=num_epochs, validation_data=(X_test_texts, y_test))

# Make predictions in smaller batches for faster runtime
batch_size = 32  # You can adjust this batch size
predictions = model.predict(X_test_texts, batch_size=batch_size)

# Process predictions
predicted_classes = np.argmax(predictions, axis=1)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, predicted_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")


Epoch 1/2



1850/1850 [==============================] - 389s 210ms/step - loss: 1.1259 - accuracy: 0.5254 - val_loss: 0.9566 - val_accuracy: 0.6025
Epoch 2/2
463/463 [==============================] - 7s 14ms/step
Accuracy: 74.36%


Fine tuning with user-controlled preprocessing

In [10]:
import re
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics import accuracy_score

def custom_text_preprocessing(text):
    # Lowercasing
    text = text.lower()
    
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    return text

# Apply custom preprocessing to the text data
df['text'] = df['text'].apply(custom_text_preprocessing)

# You might want to reinitialize your tokenizer if you want to start afresh
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

# Pad sequences for uniform length
max_sequence_length = max([len(x) for x in sequences])
X = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, encoded_labels, test_size=0.2, random_state=42)

# Convert sequences to text for training and testing
X_train_texts = np.array([sequence_to_text(seq) for seq in X_train])
X_test_texts = np.array([sequence_to_text(seq) for seq in X_test])

# Train the model with the custom preprocessed data
model.fit(X_train_texts, y_train, epochs=num_epochs, validation_data=(X_test_texts, y_test))

# Make predictions on the test set
predictions = model.predict(X_test_texts)

# Convert these predictions into class labels
predicted_classes = np.argmax(predictions, axis=1)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, predicted_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")

Epoch 1/2
1850/1850 [==============================] - 390s 211ms/step - loss: 1.2045 - accuracy: 0.4823 - val_loss: 1.0188 - val_accuracy: 0.5831
Epoch 2/2
463/463 [==============================] - 6s 13ms/step
Accuracy: 69.68%


Fine tuning a custom model

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Define the vocabulary size (you can choose a number, like the size of your tokenizer's word index)
vocab_size = len(tokenizer.word_index) + 1

# Define a simple LSTM model
custom_model = Sequential()
custom_model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_sequence_length))
custom_model.add(LSTM(128, return_sequences=True))
custom_model.add(Dropout(0.2))
custom_model.add(LSTM(64))
custom_model.add(Dense(16, activation='relu'))
custom_model.add(Dense(4, activation='softmax'))  # 4 classes

custom_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the custom model (adjust epochs and batch_size as needed)
custom_model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Make predictions
predictions = custom_model.predict(X_test)

# Convert these predictions into class labels
predicted_classes = np.argmax(predictions, axis=1)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, predicted_classes)
print(f"Custom Model Accuracy: {accuracy * 100:.2f}%")

Epoch 1/5
1850/1850 [==============================] - 172s 91ms/step - loss: 0.8635 - accuracy: 0.6531 - val_loss: 0.5770 - val_accuracy: 0.7881
Epoch 2/5
1850/1850 [==============================] - 163s 88ms/step - loss: 0.3676 - accuracy: 0.8683 - val_loss: 0.4284 - val_accuracy: 0.8472
Epoch 3/5
1850/1850 [==============================] - 159s 86ms/step - loss: 0.2249 - accuracy: 0.9178 - val_loss: 0.4065 - val_accuracy: 0.8630
Epoch 4/5
1850/1850 [==============================] - 168s 91ms/step - loss: 0.1645 - accuracy: 0.9372 - val_loss: 0.3996 - val_accuracy: 0.8701
Epoch 5/5
463/463 [==============================] - 13s 26ms/step
Custom Model Accuracy: 87.78%
